In [107]:
# Wikipedia
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

def search_wikipedia(query):
    api_wrapper = WikipediaAPIWrapper(top_k_results=2)
    tool = WikipediaQueryRun(api_wrapper=api_wrapper)
    return tool._run(query)

In [108]:
# DuckDuckGo
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.document_loaders import WebBaseLoader


def search_duckduckgo(query):
    duckduck_api_wrapper = DuckDuckGoSearchAPIWrapper(time="m")
    docs = duckduck_api_wrapper.results(query, max_results=1)
    links = []
    for doc in docs:
        links.append(doc["link"])

    loader = WebBaseLoader(
        web_path=links,
        bs_get_text_kwargs={
            "strip": True,
        }
    )

    load = loader.load()
    return [content.page_content for content in load]

In [109]:
# save .txt file
def save_file(results):
    with open("duckduckgo_result.txt", 'w', encoding="utf-8") as f:
        for result in results:
            f.write(result.metadata['title'] + "\n")
            f.write("--------------------------------------\n")
            f.write(result.page_content + "\n")
            f.write("--------------------------------------\n")

In [110]:
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from datetime import datetime

load_dotenv()

llm = ChatOpenAI()


class SearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search"
    )


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "query에 대한 답을 Wikipedia를 활용해 검색합니다."
    args_schema = SearchToolArgsSchema

    def _run(self, query):
        return search_wikipedia(query)


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "query에 대한 답을 DuckDuckGo를 활용해 검색합니다."
    args_schema = SearchToolArgsSchema

    def _run(self, query):
        return search_duckduckgo(query)


class FileSaveArgsSchema(BaseModel):
    content: str = Field(
        description="검색 도구들로부터 수집한 전체 리서치 내용과 결과를 여기에 전달하세요. 파일명이 아닌 실제 조사한 내용 전체를 포함해야 합니다."
    )


class FileSaveTool(BaseTool):
    name = "FileSaveTool"
    description = """
    리서치 작업이 완료되면 반드시 사용해야 하는 도구입니다.
    Wikipedia와 DuckDuckGo에서 수집한 모든 정보를 정리하여 content 파라미터에 전달하면 자동으로 파일로 저장됩니다.
    주의: content에는 파일명이 아닌, 실제로 조사한 내용 요약/정리해서 넣어야 합니다.
    """
    args_schema = FileSaveArgsSchema
    return_direct = True

    def _run(self, content):
        now = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
        filename = f"research_result_{now}.txt"
        with open(filename, 'w', encoding="utf-8") as f:
            f.write(content + "\n")
        return content


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
        FileSaveTool(),
    ],
)

response = agent.invoke("Research about the XZ backdoor")
response



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `XZ backdoor`


['XZ Utils backdoor - WikipediaJump to contentMain menuMain menumove to sidebarhideNavigationMain pageContentsCurrent eventsRandom articleAbout WikipediaContact usContributeHelpLearn to editCommunity portalRecent changesUpload fileSpecial pagesSearchSearchAppearanceDonateCreate accountLog inPersonal toolsDonateCreate accountLog inPages for logged out editorslearn moreContributionsTalkContentsmove to sidebarhide(Top)1Background2Mechanism3ResponseToggle Response subsection3.1Remediation3.2Broader response4Notes5References6External linksToggle the table of contentsXZ Utils backdoor12 languagesCatalàČeštinaDeutschEspañolFrançais한국어日本語РусскийShqipSimple EnglishУкраїнська中文Edit linksArticleTalkEnglishReadEditView historyToolsToolsmove to sidebarhideActionsReadEditView historyGeneralWhat links hereRelated changesUpload filePermanent linkPage informationCite this pageGet shortened URLDownload QR code

{'input': 'Research about the XZ backdoor',
 'output': 'In February 2024, a malicious backdoor was introduced to the Linux build of the XZ utility within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name "Jia Tan". The backdoor allows an attacker to execute remote code through OpenSSH on the affected Linux system. The issue was assigned the Common Vulnerabilities and Exposures number CVE-2024-3094 and has a CVSS score of 10.0. The backdoor was discovered by software developer Andres Freund. The attacker made efforts to obfuscate the code, and the backdoor was notable for its level of sophistication. The mechanism of the backdoor involves replacing an existing function in OpenSSH with a malicious version. Remediation efforts included rolling back to previous uncompromised versions and reverting affected packages to older versions by Linux software vendors. The incident raised concerns about the reliance on unpaid volunteers in critical cyber infrastructure.'}